In [1]:
import sys
import os
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
from Src.config import hf_access_token, hf_model_cache_dir # noqa: E402
os.environ["HF_HOME"] = hf_model_cache_dir
import torch

%load_ext autoreload
%autoreload 2

sim_data = torch.load('../data/full_data_sampled_gpt2_similarity_self-similarity_10000.pt')

In [2]:
from Src.model import ModelFactory
model = ModelFactory.create("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
from Src.dataset import BaseDataset

dataset = BaseDataset(path = "../data/full_data_sampled_gpt2.json",
                      model = model,
                      experiment="copyVSfact",
                      no_subject=True)

 No subject found in the dataset , proceeding with no subject data


Tokenizing and computing lengths: 100%|██████████| 10000/10000 [00:05<00:00, 1921.72it/s]


In [12]:
dataset[0]["input_ids"].shape[0]


24

In [15]:
# count the average number of tokens for each sentence in the dataset
lengths = dataset.get_lengths()
sentence_tokens = []
for l in lengths:
    dataset.set_len(l)
    for i in range(len(dataset)):
        sentence_tokens.append(dataset[i]["input_ids"].shape[0])

In [20]:
sentence_tokens.sum()/len(sentence_tokens)

tensor(21.0506)

In [23]:
(sentence_tokens**2).sum()/len(sentence_tokens)

tensor(464.8708)

In [16]:
sentence_tokens = torch.tensor(sentence_tokens)
print(sentence_tokens.mean().item())


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_313967/2704960381.py:2 in <module>                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_313967/2704960381.py'                │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: mean(): could not infer output dtype. Input dtype must be either a floating 
point or complex dtype. Got: Long

In [34]:
from Src.experiment import Ablator
ablator = Ablator(model=model, dataset=dataset, experiment="copyVSfact", batch_size=20)


In [52]:
ablator.set_heads(heads=[(10,7), (11,2)], value=5)
ablator.run()

Ablating: 100%|██████████| 24/24 [00:38<00:00,  1.60s/it]


,hooks,mem,cp,diff,mem_std,cp_std,diff_std,mem_win,cp_win
0,blocks.10.attn.hook_pattern,14.633858,17.020144,-2.386285,1.772223,4.101615,3.899835,2055.0,7319.0
1,blocks.11.attn.hook_pattern,14.633858,17.020144,-2.386285,1.772223,4.101615,3.899835,2055.0,7319.0


In [24]:
cache = model.run_with_cache("ciao questa è una prova")

In [26]:
cache[1]

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_re